In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold, train_test_split
import xgboost as xgb

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
data = pd.read_csv("./data/competition_data.csv")

In [4]:
data.drop(columns=['benefit', 'deal_print_id','etl_version', 'full_name', 
                   'warranty', 'item_id', 'main_picture',
                    'site_id','uid', 'user_id', 'category_id', 'title', 'tags'], axis= "columns", inplace=True)

data['is_pdp'] = data['is_pdp'].astype('bool')

Variables con texto: full_name, warranty, tags, title, category_id

Convertir: print_server_timestamp


In [5]:
data.drop(columns=["print_server_timestamp", 'date', 'platform', "domain_id"], axis= "columns", inplace=True)

In [6]:
data = pd.get_dummies(data,columns = ["listing_type_id", "logistic_type" ],dummy_na = False, dtype = int )

In [7]:
train_data = data[data["ROW_ID"].isna()]
test_data = data[data["ROW_ID"].notna()]

x_train = train_data.drop(columns=["conversion", "ROW_ID"])
y_train = train_data["conversion"]
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=3456)
X_test = test_data.drop(columns=["conversion"])

In [8]:
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic',
                            seed = 100,
                            eval_metric = 'auc')

In [9]:
clf_xgb.fit(X_train, Y_train, verbose = True, eval_set = [(X_val, Y_val)])

/Users/tomasglauberman/opt/anaconda3/envs/td6/lib/python3.11/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/tomasglauberman/opt/anaconda3/envs/td6/lib/python3.11/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/Users/tomasglauberman/opt/anaconda3/envs/td6/lib/python3.11/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/Users/tomasglauberman/opt/anaconda3/envs/td6/lib/python3.11/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. U

[0]	validation_0-auc:0.85938
[1]	validation_0-auc:0.86362
[2]	validation_0-auc:0.86524
[3]	validation_0-auc:0.86623
[4]	validation_0-auc:0.86672
[5]	validation_0-auc:0.86772
[6]	validation_0-auc:0.86786
[7]	validation_0-auc:0.86836
[8]	validation_0-auc:0.86937
[9]	validation_0-auc:0.86974
[10]	validation_0-auc:0.87112
[11]	validation_0-auc:0.87155
[12]	validation_0-auc:0.87168
[13]	validation_0-auc:0.87301
[14]	validation_0-auc:0.87396
[15]	validation_0-auc:0.87449
[16]	validation_0-auc:0.87476
[17]	validation_0-auc:0.87495
[18]	validation_0-auc:0.87552
[19]	validation_0-auc:0.87585
[20]	validation_0-auc:0.87596
[21]	validation_0-auc:0.87607
[22]	validation_0-auc:0.87639
[23]	validation_0-auc:0.87646
[24]	validation_0-auc:0.87680
[25]	validation_0-auc:0.87698
[26]	validation_0-auc:0.87735
[27]	validation_0-auc:0.87760
[28]	validation_0-auc:0.87779
[29]	validation_0-auc:0.87792
[30]	validation_0-auc:0.87799
[31]	validation_0-auc:0.87821
[32]	validation_0-auc:0.87829
[33]	validation_0-au

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [10]:
y_preds = clf_xgb.predict_proba(X_test.drop(columns=["ROW_ID"]))[:, clf_xgb.classes_ == 1].squeeze()

/Users/tomasglauberman/opt/anaconda3/envs/td6/lib/python3.11/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/tomasglauberman/opt/anaconda3/envs/td6/lib/python3.11/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/Users/tomasglauberman/opt/anaconda3/envs/td6/lib/python3.11/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/Users/tomasglauberman/opt/anaconda3/envs/td6/lib/python3.11/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. U

In [11]:
submission_df = pd.DataFrame({"ROW_ID": X_test["ROW_ID"], "conversion": y_preds})
submission_df["ROW_ID"] = submission_df["ROW_ID"].astype(int)
submission_df.to_csv("./outputs/primer_modelo_limp_data.csv", sep=",", index=False)